# Capture Rate Analysis
Calculate TRUE capture rate for each trade by reconstructing daily P&L trajectories.

**Goal:** For each trade, find peak P&L and compare to exit P&L.

In [ ]:
from AlgorithmImports import *
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

qb = QuantBook()

## 1. Load Trades from Backtest Orders

In [ ]:
# Load trades directly from backtest orders API
import requests
import hashlib
import time

USER_ID = "444200"
API_TOKEN = "8dce7bf6c29a042ee5700bcbeaaf89dcf54865d2c3b7bab5513436f7baba70c7"
PROJECT_ID = "26652712"
BACKTEST_ID = "af19bbcc634fd514edf471b6af4eac4c"  # Lifecycle backtest

timestamp = str(int(time.time()))
hashed = hashlib.sha256(f"{API_TOKEN}:{timestamp}".encode('utf-8')).hexdigest()
auth = (USER_ID, hashed)
headers = {"Timestamp": timestamp}

# Fetch all orders
all_orders = []
start = 0
while True:
    params = {"projectId": PROJECT_ID, "backtestId": BACKTEST_ID, "start": start, "end": start + 100}
    resp = requests.get("https://www.quantconnect.com/api/v2/backtests/orders/read", 
                        params=params, auth=auth, headers=headers)
    orders = resp.json().get("orders", [])
    if not orders: break
    all_orders.extend(orders)
    start += 100
    if len(orders) < 100: break

# Reconstruct trades
positions = {}
trades_list = []
for order in sorted(all_orders, key=lambda x: x.get("time", "")):
    symbol_data = order.get("symbol", {})
    full_symbol = symbol_data.get("value", "")
    underlying = symbol_data.get("underlying", {}).get("value", full_symbol.split()[0])
    qty, price = order.get("quantity", 0), order.get("price", 0)
    order_time = order.get("time", "")[:10]
    value = abs(qty * price * 100)
    
    if qty > 0:
        if underlying not in positions:
            positions[underlying] = {"entry_date": order_time, "entry_cost": 0}
        positions[underlying]["entry_cost"] += value
    elif qty < 0 and underlying in positions:
        pos = positions[underlying]
        if "exit_value" not in pos:
            pos["exit_value"], pos["exit_date"] = 0, order_time
        pos["exit_value"] += value
        if "C0" in full_symbol: pos["call_closed"] = True
        elif "P0" in full_symbol: pos["put_closed"] = True
        if pos.get("call_closed") and pos.get("put_closed"):
            pnl_pct = (pos["exit_value"] / pos["entry_cost"] - 1) * 100 if pos["entry_cost"] > 0 else 0
            trades_list.append({"symbol": underlying, "entry_date": pos["entry_date"],
                "exit_date": pos["exit_date"], "entry_cost": pos["entry_cost"], "pnl_pct": pnl_pct})
            del positions[underlying]

trades = pd.DataFrame(trades_list)
print(f"Loaded {len(trades)} trades from backtest")
trades.head()

## 2. Function to Get Daily Straddle Prices

In [ ]:
def get_straddle_trajectory(qb, symbol, entry_date, exit_date, entry_cost):
    """
    Get daily P&L trajectory for a straddle.
    Uses QuantBook to fetch historical option prices.
    """
    start = pd.Timestamp(entry_date)
    end = pd.Timestamp(exit_date)
    
    # Add equity
    equity = qb.AddEquity(symbol, Resolution.Daily)
    
    # Get underlying price history
    history = qb.History(equity.Symbol, start, end + timedelta(days=1), Resolution.Daily)
    
    if history.empty:
        return None
    
    # Get option chain for the entry date to find ATM strike
    contracts = qb.OptionChainProvider.GetOptionContractList(equity.Symbol, start)
    
    # Find the ATM straddle contracts
    spot = history.iloc[0]['close']
    
    # Filter to ~45-70 DTE, ATM
    valid_contracts = []
    for contract in contracts:
        dte = (contract.ID.Date - start).days
        if 45 <= dte <= 70:
            valid_contracts.append(contract)
    
    if not valid_contracts:
        return None
    
    # Find ATM strike
    strikes = set(c.ID.StrikePrice for c in valid_contracts)
    atm_strike = min(strikes, key=lambda s: abs(s - spot))
    
    # Get call and put at ATM strike
    call_contract = next((c for c in valid_contracts if c.ID.StrikePrice == atm_strike and c.ID.OptionRight == OptionRight.Call), None)
    put_contract = next((c for c in valid_contracts if c.ID.StrikePrice == atm_strike and c.ID.OptionRight == OptionRight.Put), None)
    
    if not call_contract or not put_contract:
        return None
    
    # Get price history for both contracts
    call_history = qb.History(call_contract, start, end + timedelta(days=1), Resolution.Daily)
    put_history = qb.History(put_contract, start, end + timedelta(days=1), Resolution.Daily)
    
    if call_history.empty or put_history.empty:
        return None
    
    # Calculate daily straddle value and P&L
    trajectory = []
    peak_pnl = 0
    peak_date = start
    
    for date in pd.date_range(start, end):
        try:
            call_price = call_history.loc[date]['close'] if date in call_history.index else None
            put_price = put_history.loc[date]['close'] if date in put_history.index else None
            
            if call_price and put_price:
                straddle_value = (call_price + put_price) * 100  # Per contract
                pnl_pct = (straddle_value / entry_cost - 1) * 100
                
                if pnl_pct > peak_pnl:
                    peak_pnl = pnl_pct
                    peak_date = date
                
                trajectory.append({
                    'date': date,
                    'pnl_pct': pnl_pct,
                    'peak_pnl_pct': peak_pnl,
                    'peak_date': peak_date
                })
        except:
            continue
    
    return pd.DataFrame(trajectory)

## 3. Analyze All Trades

In [ ]:
results = []

for idx, trade in trades.iterrows():
    print(f"Analyzing {trade['symbol']} {trade['entry_date']}...")
    
    trajectory = get_straddle_trajectory(
        qb, 
        trade['symbol'], 
        trade['entry_date'], 
        trade['exit_date'],
        trade['entry_cost']
    )
    
    if trajectory is not None and len(trajectory) > 0:
        peak_row = trajectory.iloc[-1]  # Last row has running peak
        peak_pnl = peak_row['peak_pnl_pct']
        peak_date = peak_row['peak_date']
        exit_pnl = trade['pnl_pct']
        
        if peak_pnl > 0:
            capture_rate = (exit_pnl / peak_pnl) * 100
        else:
            capture_rate = 100 if exit_pnl <= 0 else 0
        
        results.append({
            'symbol': trade['symbol'],
            'entry_date': trade['entry_date'],
            'exit_date': trade['exit_date'],
            'exit_pnl_pct': exit_pnl,
            'peak_pnl_pct': peak_pnl,
            'peak_date': str(peak_date)[:10],
            'capture_rate': capture_rate,
            'days_from_peak': (pd.Timestamp(trade['exit_date']) - peak_date).days
        })

results_df = pd.DataFrame(results)
print(f"\nAnalyzed {len(results_df)} trades")
results_df.head(10)

## 4. Capture Rate Summary

In [ ]:
# Filter to trades with positive peaks
with_peaks = results_df[results_df['peak_pnl_pct'] > 0]

print("="*60)
print("CAPTURE RATE ANALYSIS")
print("="*60)
print(f"\nTrades with positive peaks: {len(with_peaks)} / {len(results_df)}")
print(f"\nTotal Peak P&L available:  {with_peaks['peak_pnl_pct'].sum():.1f}%")
print(f"Total Exit P&L captured:   {with_peaks['exit_pnl_pct'].sum():.1f}%")
print(f"\nOVERALL CAPTURE RATE: {with_peaks['exit_pnl_pct'].sum() / with_peaks['peak_pnl_pct'].sum() * 100:.0f}%")
print(f"\nAvg days from peak to exit: {with_peaks['days_from_peak'].mean():.1f}")

In [ ]:
# Export results
results_df.to_csv('capture_rate_results.csv', index=False)
print("Saved to capture_rate_results.csv")